## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/model_fold_1.h5
/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/model_fold_2.h5
/kaggle/input/141-googleq-a-train-3fold-bert-base-unc-regression/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output = bert_model([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="linear", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)

submission = submission.clip(0, 1)

display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,1,0.960719,0.647934,0.273638,0.322171,0.601395,0.584020,0.708393,0.694120,0.646573,...,0.926749,0.855709,0.554173,0.964068,0.946265,0.794137,0.081295,0.026803,0.803767,0.958685
1,1,0.876690,0.462229,0.012126,0.699632,0.770134,0.934272,0.572890,0.456616,0.129165,...,0.706638,0.930395,0.617705,0.987177,0.963918,0.865317,0.863224,0.060073,0.201122,0.901471
2,1,0.918617,0.667303,0.044137,0.680471,0.846264,0.936572,0.638638,0.532878,0.168873,...,0.906015,0.902754,0.621856,0.974505,0.964959,0.870375,0.103927,0.027272,0.829011,0.920309
3,1,0.850678,0.399678,0.043251,0.648295,0.736034,0.898279,0.563808,0.451085,0.098484,...,0.670195,0.944324,0.678333,1.000000,1.000000,0.937688,0.784087,0.116373,0.569759,0.928105
4,1,0.956691,0.476484,0.103806,0.778467,0.732745,0.828088,0.689064,0.639738,0.177503,...,0.749077,0.899274,0.661653,0.988001,0.931597,0.856331,0.201460,0.080956,0.704737,0.946534


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.0,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,1.0,0.893963,0.575157,0.059015,0.641571,0.782990,0.840322,0.605097,0.517758,0.234915,...,0.821304,0.906961,0.647598,0.961028,0.972553,0.865193,0.500425,0.077147,0.526895,0.915232
std,0.0,0.049463,0.123432,0.079724,0.108923,0.105508,0.124589,0.047840,0.081711,0.182940,...,0.081197,0.028418,0.044868,0.020291,0.021927,0.041848,0.311461,0.045027,0.246911,0.024954
min,1.0,0.773157,0.290550,0.000000,0.242076,0.190393,0.187643,0.504558,0.366077,0.000000,...,0.636581,0.817101,0.522415,0.901604,0.892871,0.749491,0.000000,0.000000,0.049112,0.828525
25%,1.0,0.857308,0.470755,0.011214,0.578881,0.733706,0.798622,0.572589,0.460044,0.094497,...,0.753179,0.887657,0.618062,0.946527,0.960105,0.837885,0.197828,0.044729,0.340102,0.897159
50%,1.0,0.891582,0.571963,0.029973,0.651107,0.789457,0.874725,0.594628,0.493405,0.183971,...,0.824514,0.906457,0.646142,0.962769,0.975566,0.866082,0.558522,0.079173,0.522049,0.914036
75%,1.0,0.929356,0.673570,0.070082,0.709179,0.845751,0.921426,0.635680,0.553400,0.336144,...,0.890893,0.927631,0.675993,0.974578,0.990769,0.895524,0.779552,0.109769,0.724304,0.931970
max,1.0,1.000000,0.858653,0.612068,0.965998,1.000000,1.000000,0.738713,0.854713,0.795261,...,1.000000,0.992529,0.795043,1.000000,1.000000,0.994034,0.977490,0.204425,1.000000,0.995704
